In [2]:
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objs as go

# Load the dataset
file_path = 'C:\\Users\\loydt\\Downloads\\Projects\\Superstore Sales Dataset.xlsx'
data = pd.read_excel(file_path)

# Convert the 'Order Date' column to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d/%m/%Y', errors='coerce')

# Set 'Order Date' as the index
data.set_index('Order Date', inplace=True)

# List of unique sub-categories
sub_categories = ['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage', 'Furnishings', 
                  'Art', 'Phones', 'Binders', 'Appliances', 'Paper', 'Accessories', 
                  'Envelopes', 'Fasteners', 'Supplies', 'Machines', 'Copiers']

# Loop through each sub-category to create individual charts
for sub_category in sub_categories:
    # Filter data for the current sub-category
    sub_category_data = data[data['Sub-Category'] == sub_category]
    
    # Resample the data to get monthly sales
    monthly_sales = sub_category_data['Sales'].resample('ME').sum()
    
    # Perform seasonal decomposition
    decomposition = sm.tsa.seasonal_decompose(monthly_sales, model='additive', period=12)
    
    # Extract the seasonal and residual components
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    # Print the dates with corresponding seasonal values for performance evaluation
    print(f"\nSales Performance Evaluation for {sub_category}:")
    
    # Initialize lists to collect expected increases, drops, exceeded expectations, and below expected drops
    expected_increases = []
    expected_drops = []
    expected_increases_exceeded = []
    below_expected_drops = []

    # Initialize totals for each category
    total_failed_expectations = 0
    total_exceeded_expectations = 0
    total_below_expected_drops = 0
    total_drop_exceeded_expectations = 0
    
    for date, seasonal_value, residual_value in zip(seasonal.index, seasonal, residual):
        if pd.notna(residual_value):  # Check if the residual value is not NaN
            if seasonal_value < 0:
                # Collect negative seasonal values
                expected_drops.append((date.date(), seasonal_value))
            elif seasonal_value > 0:
                # Collect positive seasonal values
                expected_increases.append((date.date(), seasonal_value))

    # Prepare recorded values for residuals
    recorded_values = {}
    
    for date, residual_value in zip(residual.index, residual):
        if pd.notna(residual_value):  # Check if the residual value is not NaN
            recorded_values[date.date()] = residual_value

    # Print expected increases and corresponding recorded values
    print("\nFor Expected Increase but Failed:")
    print("Date | Expected Increase | Recorded Value | Failed Expectations")
    
    for date, expected_value in expected_increases:
        recorded_value = recorded_values.get(date)
        if recorded_value is not None:
            if expected_value > 0 and recorded_value < expected_value:
                failed_expectations = abs(expected_value - recorded_value)
                total_failed_expectations += failed_expectations
                print(f"{date} | {expected_value:.2f} | {recorded_value:.2f} | {failed_expectations:.2f}")
            elif expected_value > 0 and recorded_value > expected_value:
                exceeded_expectations = abs(recorded_value - expected_value)
                total_exceeded_expectations += exceeded_expectations
                expected_increases_exceeded.append((date, expected_value, recorded_value, exceeded_expectations))

    # Print expected increases and exceeded expectations
    print("\nFor Expected Increase and Exceeded:")
    print("Date | Expected Increase | Recorded Value | Exceeded Expectations")

    for date, expected_value, recorded_value, exceeded_expectations in expected_increases_exceeded:
        print(f"{date} | {expected_value:.2f} | {recorded_value:.2f} | {exceeded_expectations:.2f}")

    # Print expected drops and corresponding recorded values for Below Expected Drop list
    print("\nBelow Expected Drop:")
    print("Date | Expected Drop | Recorded Value | Failed Expectations")
    
    for date, expected_value in expected_drops:
        recorded_value = recorded_values.get(date)
        if recorded_value is not None:
            if recorded_value < expected_value:
                failed_expectations = abs(expected_value - recorded_value)
                total_below_expected_drops += failed_expectations
                below_expected_drops.append((date, expected_value, recorded_value, failed_expectations))
                print(f"{date} | {expected_value:.2f} | {recorded_value:.2f} | {failed_expectations:.2f}")

    # Print expected drops and corresponding recorded values
    print("\nFor Expected Drop but Exceeded:")
    print("Date | Expected Drop | Recorded Value | Exceeded Expectations")

    for date, expected_value in expected_drops:
        recorded_value = recorded_values.get(date)
        if recorded_value is not None:
            exceeded_expectations = abs(max(expected_value, recorded_value) - min(expected_value, recorded_value))
            total_drop_exceeded_expectations += exceeded_expectations
            print(f"{date} | {expected_value:.2f} | {recorded_value:.2f} | {exceeded_expectations:.2f}")

    # Print total values for each category
    print(f"\nTotal for {sub_category}:")
    print(f"Expected Increase but Failed Total: {total_failed_expectations:.2f}")
    print(f"Expected Increase and Exceeded Total: {total_exceeded_expectations:.2f}")
    print(f"Below Expected Drop Total: {total_below_expected_drops:.2f}")
    print(f"Expected Drop but Exceeded Total: {total_drop_exceeded_expectations:.2f}")

    # Create a line plot for seasonal and residual components
    fig = go.Figure()

    # Add seasonal component
    fig.add_trace(go.Scatter(
        x=seasonal.index,
        y=seasonal,
        mode='lines',
        name='Seasonal Component',
        line=dict(width=2)
    ))

    # Add residual component with custom color and line style
    fig.add_trace(go.Scatter(
        x=residual.index,
        y=residual,
        mode='lines',
        name='Residual Component',
        line=dict(width=2, dash='dash', color='orange')
    ))

    # Update layout with dark theme
    fig.update_layout(
        title=f'Seasonal and Residual Components for {sub_category}',
        xaxis_title='Date',
        yaxis_title='Value',
        legend_title='Components',
        template='plotly_dark'
    )

    # Show the figure
    fig.show()



Sales Performance Evaluation for Bookcases:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-09-30 | 4141.00 | -3528.45 | 7669.45
2015-11-30 | 4388.03 | 971.00 | 3417.03
2016-06-30 | 103.59 | -2760.47 | 2864.05
2016-09-30 | 4141.00 | 3436.51 | 704.49
2016-11-30 | 4388.03 | 269.23 | 4118.81
2017-09-30 | 4141.00 | 379.52 | 3761.48
2017-11-30 | 4388.03 | -952.65 | 5340.68

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2017-06-30 | 103.59 | 259.69 | 156.10
2018-06-30 | 103.59 | 2788.35 | 2684.77

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2016-07-31 | -117.20 | -163.59 | 46.39
2016-12-31 | -195.33 | -197.36 | 2.03
2017-05-31 | -872.52 | -1482.43 | 609.91
2017-08-31 | -698.70 | -803.82 | 105.12
2017-12-31 | -195.33 | -818.41 | 623.08

For Expected Drop but Exceeded:
Date | Expected Drop | Recorded Value | Exceeded Expectations
2015-07-31 | -11


Sales Performance Evaluation for Chairs:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-09-30 | 4755.73 | 2658.79 | 2096.94
2015-11-30 | 4624.72 | -3881.03 | 8505.75
2015-12-31 | 7635.22 | 3082.19 | 4553.03
2016-05-31 | 633.58 | -1191.84 | 1825.42
2016-09-30 | 4755.73 | -1730.55 | 6486.28
2016-11-30 | 4624.72 | 1821.76 | 2802.96
2016-12-31 | 7635.22 | -3672.19 | 11307.40
2017-05-31 | 633.58 | 345.75 | 287.83
2017-09-30 | 4755.73 | -977.37 | 5733.10
2017-11-30 | 4624.72 | 2010.14 | 2614.58
2017-12-31 | 7635.22 | 540.86 | 7094.36

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2018-05-31 | 633.58 | 796.95 | 163.37

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2017-07-31 | -120.99 | -1768.92 | 1647.93
2017-10-31 | -1.49 | -838.69 | 837.19
2018-04-30 | -1591.50 | -4009.88 | 2418.38

For Expected Drop but Exceeded:
Date | Expected Drop | Record


Sales Performance Evaluation for Labels:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-07-31 | 15.85 | -56.66 | 72.51
2015-09-30 | 79.77 | 37.38 | 42.39
2015-10-31 | 52.18 | -234.16 | 286.35
2015-11-30 | 318.59 | 1.73 | 316.85
2015-12-31 | 138.08 | 104.63 | 33.45
2016-09-30 | 79.77 | -70.61 | 150.38
2016-11-30 | 318.59 | -374.46 | 693.05
2016-12-31 | 138.08 | 14.32 | 123.76
2017-07-31 | 15.85 | -57.84 | 73.69
2017-09-30 | 79.77 | -13.95 | 93.73
2017-10-31 | 52.18 | -157.79 | 209.97
2017-12-31 | 138.08 | -166.13 | 304.22

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2016-07-31 | 15.85 | 67.31 | 51.46
2016-10-31 | 52.18 | 344.77 | 292.58
2017-11-30 | 318.59 | 325.54 | 6.95

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2015-08-31 | -60.59 | -140.62 | 80.04
2016-05-31 | -2.88 | -63.06 | 60.18
2016-06-30 | -55.31 | -139.64 | 84.33
2017-03-31


Sales Performance Evaluation for Tables:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-08-31 | 394.07 | -1662.63 | 2056.70
2015-09-30 | 876.97 | -217.11 | 1094.08
2015-11-30 | 2781.89 | -1816.98 | 4598.87
2015-12-31 | 6356.49 | -1383.32 | 7739.81
2016-09-30 | 876.97 | 30.59 | 846.38
2016-11-30 | 2781.89 | 301.58 | 2480.32
2016-12-31 | 6356.49 | -2550.50 | 8906.99
2017-09-30 | 876.97 | -253.02 | 1129.99
2017-11-30 | 2781.89 | 1075.86 | 1706.03
2017-12-31 | 6356.49 | 3494.29 | 2862.20

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2016-08-31 | 394.07 | 451.72 | 57.65
2017-08-31 | 394.07 | 771.37 | 377.30

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2016-06-30 | -617.84 | -1574.12 | 956.28
2017-01-31 | -153.16 | -957.38 | 804.22
2017-10-31 | -796.88 | -2107.76 | 1310.87
2018-01-31 | -153.16 | -1946.87 | 1793.71
2018-06-30 | -617.84 | -951.


Sales Performance Evaluation for Storage:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-09-30 | 3873.13 | 1060.60 | 2812.52
2015-11-30 | 3974.64 | 3595.54 | 379.10
2015-12-31 | 1976.34 | -924.67 | 2901.01
2016-05-31 | 13.06 | -779.08 | 792.14
2016-09-30 | 3873.13 | 1242.50 | 2630.62
2016-11-30 | 3974.64 | -1553.34 | 5527.98
2016-12-31 | 1976.34 | -2465.58 | 4441.92
2017-09-30 | 3873.13 | -2096.96 | 5970.09
2017-11-30 | 3974.64 | -1836.06 | 5810.70
2018-05-31 | 13.06 | -831.29 | 844.35

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2017-05-31 | 13.06 | 1816.51 | 1803.45
2017-12-31 | 1976.34 | 3596.39 | 1620.05

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2015-08-31 | -559.37 | -1042.52 | 483.15
2016-04-30 | -869.33 | -1080.70 | 211.37
2016-06-30 | -17.82 | -816.18 | 798.35
2017-06-30 | -17.82 | -877.90 | 860.08
2017-08-31 | -559.37 | -18

...


Sales Performance Evaluation for Envelopes:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-07-31 | 21.48 | -197.63 | 219.11
2015-09-30 | 162.23 | -111.01 | 273.24
2015-11-30 | 435.24 | 312.81 | 122.43
2015-12-31 | 282.12 | -145.45 | 427.58
2016-03-31 | 68.06 | -256.17 | 324.23
2016-09-30 | 162.23 | 72.72 | 89.50
2016-11-30 | 435.24 | -315.87 | 751.11
2016-12-31 | 282.12 | 254.13 | 27.99
2017-09-30 | 162.23 | 48.32 | 113.91
2017-11-30 | 435.24 | 13.10 | 422.14
2017-12-31 | 282.12 | -98.65 | 380.77

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2016-07-31 | 21.48 | 109.82 | 88.34
2017-03-31 | 68.06 | 120.16 | 52.10
2017-07-31 | 21.48 | 97.84 | 76.36
2018-03-31 | 68.06 | 146.05 | 77.99

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2016-08-31 | -172.51 | -210.13 | 37.62
2017-05-31 | -59.55 | -205.71 | 146.15
2017-10-31 | -38.08 | -136.06 | 97


Sales Performance Evaluation for Fasteners:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-09-30 | 22.25 | 20.91 | 1.35
2015-10-31 | 25.75 | -23.95 | 49.70
2015-11-30 | 79.14 | -58.54 | 137.68
2015-12-31 | 26.45 | 4.05 | 22.41
2016-09-30 | 22.25 | -8.78 | 31.03
2016-10-31 | 25.75 | -57.74 | 83.50
2016-11-30 | 79.14 | 19.80 | 59.34
2016-12-31 | 26.45 | 15.31 | 11.15
2017-09-30 | 22.25 | -15.34 | 37.59
2017-11-30 | 79.14 | 35.53 | 43.61
2017-12-31 | 26.45 | -22.57 | 49.02

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2017-10-31 | 25.75 | 78.48 | 52.73

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2015-07-31 | -2.79 | -26.18 | 23.39
2016-02-29 | -8.56 | -31.12 | 22.56
2016-08-31 | -1.09 | -25.06 | 23.97
2017-03-31 | -24.37 | -28.72 | 4.34
2017-05-31 | -28.34 | -28.69 | 0.35
2017-08-31 | -1.09 | -13.37 | 12.27
2018-03-31 | -24.37 | -25.81 | 


Sales Performance Evaluation for Supplies:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2016-01-31 | 809.18 | -1245.07 | 2054.25
2016-03-31 | 2926.68 | -3093.10 | 6019.78
2016-04-30 | 11.08 | -196.63 | 207.71
2017-01-31 | 809.18 | -1869.68 | 2678.86
2018-03-31 | 2926.68 | -2260.47 | 5187.14
2018-04-30 | 11.08 | -365.20 | 376.28

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2017-03-31 | 2926.68 | 4881.10 | 1954.43
2017-04-30 | 11.08 | 89.36 | 78.28
2018-01-31 | 809.18 | 2642.29 | 1833.11

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2015-09-30 | -356.37 | -433.82 | 77.45
2015-12-31 | -187.21 | -644.12 | 456.91
2016-12-31 | -187.21 | -786.79 | 599.58
2017-05-31 | -324.08 | -525.28 | 201.20
2017-07-31 | -456.04 | -669.77 | 213.73
2017-09-30 | -356.37 | -788.52 | 432.14
2018-05-31 | -324.08 | -555.54 | 231.46
2018-06-30 | -285.61 | -895.88 | 61


Sales Performance Evaluation for Machines:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-11-30 | 3457.21 | -1573.99 | 5031.20
2016-04-30 | 2330.74 | -3104.81 | 5435.54
2016-06-30 | 0.69 | -2391.69 | 2392.38
2016-09-30 | 4237.92 | -6766.57 | 11004.50
2016-11-30 | 3457.21 | 48.47 | 3408.73
2016-12-31 | 713.29 | 361.63 | 351.66
2017-04-30 | 2330.74 | 1735.21 | 595.53
2017-09-30 | 4237.92 | -6427.16 | 10665.09
2017-11-30 | 3457.21 | 2141.31 | 1315.90
2017-12-31 | 713.29 | -3133.17 | 3846.45
2018-04-30 | 2330.74 | 1985.39 | 345.35

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2015-09-30 | 4237.92 | 13809.52 | 9571.60
2015-12-31 | 713.29 | 3387.32 | 2674.04
2017-06-30 | 0.69 | 2727.62 | 2726.93
2018-06-30 | 0.69 | 279.85 | 279.16

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2016-02-29 | -1090.93 | -2747.89 | 1656.96
2016-03-31 | -572.33 | -2


Sales Performance Evaluation for Copiers:

For Expected Increase but Failed:
Date | Expected Increase | Recorded Value | Failed Expectations
2015-12-31 | 939.05 | -703.22 | 1642.28
2016-03-31 | 5384.33 | -2718.54 | 8102.87
2016-05-31 | 1581.03 | -383.14 | 1964.17
2016-10-31 | 8009.47 | -8933.63 | 16943.10
2017-03-31 | 5384.33 | -7885.12 | 13269.44
2017-12-31 | 939.05 | -3675.72 | 4614.77
2018-05-31 | 1581.03 | -2647.70 | 4228.74

For Expected Increase and Exceeded:
Date | Expected Increase | Recorded Value | Exceeded Expectations
2016-12-31 | 939.05 | 4679.17 | 3740.11
2017-05-31 | 1581.03 | 3331.07 | 1750.03
2017-10-31 | 8009.47 | 9133.78 | 1124.31
2018-03-31 | 5384.33 | 10903.89 | 5519.56

Below Expected Drop:
Date | Expected Drop | Recorded Value | Failed Expectations
2016-07-31 | -313.42 | -1908.20 | 1594.78
2017-08-31 | -1845.88 | -2184.07 | 338.19
2017-09-30 | -1141.14 | -1177.24 | 36.10
2018-04-30 | -1959.51 | -2489.61 | 530.09

For Expected Drop but Exceeded:
Date | Expected D